# Assignment 3: Machine Translation with T5

**Description:** This assignment notebook builds on the material from the
[lesson 6 notebook](https://github.com/datasci-w266/2025-spring-main/blob/master/materials/lesson_notebooks/lesson_6_Machine_Translation_With_Transformer.ipynb), in which we set up a new, very small version of a T5 encoder decoder model to train from scratch on translations from Shakespearean to Modern English. Since the model was trained from scratch, it didn't work very well. In this notebook, we'll first try to make that model work a little better, changing the model configuration and output generation parameters. Then we'll fine tune a small pre-trained T5 model on this task, to see how much better we can do with even a small pre-trained model. We'll apply several evaluation metrics, find some trade-offs, and try adding a secondary dataset to address some of the remaining challenges.

This notebook should be run on a Google Colab leveraging a GPU. By default, when you open the notebook in Colab it will try to use a GPU. Since colab is providing free access to a GPU they place constraints on that access.  Therefore you might want to turn off the GPU access (Edit -> Notebook Settings) while editing and initially debugging your code (at least the setup before you train each model). You will need a GPU to full train or evaluate each of the models. Total runtime of the entire notebook (with solutions and a Colab GPU) should be about 1-2h, but potentially more depending on how much you experiment. If Colab tells you that you have reached your GPU limit, wait 10-24 hours and you should be able to access a GPU again.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2024-fall-main/blob/master/assignment/a3/Machine_Translation_T5.ipynb)

The overall assignment structure is as follows:


0. Setup
  
  0.1 Libraries

  0.2 Data Acquisition

  0.3. Data Preparation


1. Tiny Seq2Seq Model Trained From Scratch
  
  1.1 Tokenizer and Model Setup

  1.2 Experimenting with Model Dimensions

  1.3 Text Generation Parameters

  1.4 Test Set Evaluation Metrics

2. Small Pre-Trained T5 Model

  2.1 Pre-Trained Model Setup and Tokenization

  2.2 Fine-Tuning the Pre-Trained Model

  2.3 Fine-Tuned Model Evaluation

  2.4 Style Classifier

  2.5 Revisit Decoder .Generate() Options

3. Adding Supplementary Paraphrase Dataset

  3.1 Load and preprocess the supplemental dataset

  3.2 Train T5 on Paraphrasing Task

  3.3 Fine-Tune Paraphrase-Trained Model on Main Task
  
  3.4 Paraphrase-Trained Model Evaluation

## 0. Setup

### 0.1 Libraries

In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q evaluate
!pip install -q tokenizers

In [ ]:
import re
import random
import numpy as np
from scipy.special import softmax

import torch
import transformers
import evaluate
from datasets import Dataset, load_dataset

# For from-scratch T5 model
from transformers import T5TokenizerFast, T5Config, T5ForConditionalGeneration

# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration  # this won't import twice, just noting here what's for each model

# For all T5 models
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# For style classifier model (also for evaluating the seq2seq model output)
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer

### O.2 Data Acquisition

We'll use the Shakespeare-to-Modern-English translation dataset from Lesson 6. The data includes aligned sentences from a number of plays by William Shakespeare.

The data was copied from this repo --[https://github.com/cocoxu/Shakespeare](https://github.com/cocoxu/Shakespeare) -- and consolidated into one file for easier handling.

You will to grab a copy from our git repo and import it to your Google drive.  From there you'll be able to easily load it in to a Colab notebook.

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/w266/a3/

Machine_Translation_with_T5.ipynb  Summarization_test.ipynb
SummarizationLLM_test.ipynb	   train_plays-org-mod.txt


In [ ]:
# Modify this path to the appropriate location in your Drive
text_file = '/content/drive/MyDrive/w266/a3/train_plays-org-mod.txt'

### O.3 Data Preparation

Each line contains a Shakespearean sentence and its corresponding modern English translation.

The Shakesperean sentence is the *source sequence* and modern English one is the *target sequence*.

In [ ]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
    old, mod = line.split("\t")
    old = old
    mod = mod
    text_pairs.append((old, mod))

In [ ]:
print(lines[:5])  # Print first 5 lines to verify content

["Who's there?\tWho's there?", 'Nay, answer me.\tNo, answer me.', 'Stand and unfold yourself.\tHalt, and identify yourself.', 'Long live the King!\tLong live the king!', 'Bernardo?\tBernardo?']


In [ ]:
# Look at some examples
for _ in range(5):
    print(random.choice(text_pairs))

('Get you hence, sirrah; saucy fellow, hence!', 'Get out, servant; rude fellow, go away!')
('Let their heads be slickly combed, their blue coats brushed, and their garters of an indifferent knit.', 'Tell them to slick down their hair, brush their blue coats, and make sure their socks match.')
('I am alone the villain of the earth, And feel I am so most.', 'I am the worst person alive, and I know it the best.')
('O, thou didst prophesy the time would come That I should wish for thee to help me curse That bottled spider, that foul bunch-backed toad!', 'Oh, you prophesied that the time would come when I would want your help in cursing that humpbacked spider, that hideous toad!')
('Not till you have slept.', 'Not until you pass out.')


In [ ]:
# Let's create some splits
random.shuffle(text_pairs)
num_val_samples = int(0.06 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

19088 total pairs
16798 training pairs
1145 validation pairs
1145 test pairs


Like we did in the lesson notebook, let's create a Huggingface dataset object from our data, so that it's easy to work with and pass to our model trainer.

In [ ]:
def make_dataset(pairs):
    org_texts, mod_texts = zip(*pairs)
    org_texts = list(org_texts)
    mod_texts = list(mod_texts)

    dataset = Dataset.from_dict({"shakespeare": org_texts, "modern": mod_texts})
    return dataset.shuffle()

# Make the training data
train_dataset = make_dataset(train_pairs)

# Make the validation data
val_dataset = make_dataset(val_pairs)

## 1. Tiny Seq2Seq Model Trained From Scratch

As in the lesson 6 notebook, for our first model, we'll make a new tokenizer and model based on the T5 architecture, which we'll train from scratch only on our task dataset.

### 1.1 Tokenizer and Model Setup

The easiest way to make a new tokenizer is to load an existing T5 one, then call .train_new_from_iterator(), providing our own dataset and vocab size.

In [ ]:
VOCAB_SIZE = 15000

def get_word_piece_tokenizer(text_samples, vocab_size):

    base_tokenizer = T5TokenizerFast.from_pretrained('t5-base')
    new_tokenizer = base_tokenizer.train_new_from_iterator(
        text_samples,
        vocab_size=VOCAB_SIZE
    )

    return new_tokenizer

In [ ]:
shakespeare_samples = [text_pair[0] for text_pair in train_pairs]
modern_samples = [text_pair[1] for text_pair in train_pairs]

part1_tokenizer = get_word_piece_tokenizer(shakespeare_samples + modern_samples, VOCAB_SIZE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

We'll need to preprocess the data using the tokenizer. Since our task is to translate from Shakespearean to Modern English, the Shakespeare text will be our input_ids and the Modern English will be the labels we use for training and evaluation. We'll create a function to do the tokenization, and then map it to our Huggingface datasets containing the train and validation data. We'll have the function take a tokenizer, because later we'll use a different pre-trained one.

In [ ]:
MAX_SEQUENCE_LENGTH = 40

def preprocess_translation_batch(batch_text_pairs, tokenizer, prefix=""):
    if prefix:
        batch_text_pairs["shakespeare"] = [prefix + text for text in batch_text_pairs["shakespeare"]]

    shakespeare_encoded = tokenizer.batch_encode_plus(
        batch_text_pairs["shakespeare"],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    modern_encoded = tokenizer.batch_encode_plus(
        batch_text_pairs["modern"],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    return {'input_ids': shakespeare_encoded['input_ids'],
            'labels': modern_encoded['input_ids']}

In [ ]:
train_ds_part1 = train_dataset.map(preprocess_translation_batch, batched=True,
                                   fn_kwargs={'tokenizer': part1_tokenizer})
val_ds_part1 = val_dataset.map(preprocess_translation_batch, batched=True,
                               fn_kwargs={'tokenizer': part1_tokenizer})

Map:   0%|          | 0/16798 [00:00<?, ? examples/s]

Map:   0%|          | 0/1145 [00:00<?, ? examples/s]

We'll need to create the new model from a config, specifying the model's dimensions. Then we'll need to make training arguments and trainer objects to be able to train the model. Let's create a function for each of those purposes, so that later we can use the functions to experiment with the available options.

First, make a function to create the model config and the model itself. Use the Lesson 6 notebook as a guide, and make sure to include all of the arguments that we've included in the function definition below. Those are what you'll experiment with next.

In [ ]:
from transformers import T5Config, T5ForConditionalGeneration

# Function to create a T5Config and new T5 model using all the function arguments
def create_from_scratch_model(num_layers, embed_dim, keyvalue_dim, dense_dim, num_heads):
    ### YOUR CODE HERE
    t5_config = T5Config(
        num_layers=num_layers,
        d_model=embed_dim,
        d_kv=keyvalue_dim,
        d_ff=dense_dim,
        num_heads=num_heads
    )

    t5_model = T5ForConditionalGeneration(t5_config)
    ### END YOUR CODE

    return t5_model

We'll also need to specify training arguments and a trainer for our model. Use the Seq2SeqTrainingArguments and Seq2SeqTrainer classes imported at the top of this notebook. You can use the Lesson 6 notebook as a guide for this too.

In [ ]:
from transformers import Seq2SeqTrainingArguments

# Function to specify training arguments for the model
def create_seq2seq_training_args(batch_size, num_epochs):
    ### YOUR CODE HERE
    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=2,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        remove_unused_columns=False
    )
    ### END YOUR CODE

    return training_args

In [ ]:
from transformers import T5Tokenizer, Seq2SeqTrainer

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Function to preprocess dataset and create a trainer for the model
def create_seq2seq_trainer(model, training_args, train_ds, val_ds):

    # Preprocessing function to tokenize input and labels
    def preprocess_translation_batch(batch):
        inputs = batch['shakespeare']
        targets = batch['modern']

        model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)
        labels = tokenizer(targets, padding="max_length", truncation=True, max_length=128)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Apply tokenization to the dataset
    train_ds = train_ds.map(preprocess_translation_batch, batched=True)
    val_ds = val_ds.map(preprocess_translation_batch, batched=True)

    # Create trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### 1.2: Experimenting with Model Dimensions

In the Lesson 6 Notebook, we created a very small T5-style model with just one transformer layer and smaller dimensions for some of the internal layers. Now, you'll explore these options yourself, to see if you can get the model to work a little better when trained on this task.

Without adding any additional training data, can we configure the model to perform better when trained on this task? What happens if we add another one or more transformer layers to the encoder and decoder, or make some of the internal dimensions smaller or larger?

The T5Config gives us several hyperparameters to adjust the model's parameter dimensions. You can see the available arguments and their default values in the [T5Config documentation](https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/t5#transformers.T5Config).

We'll give you the batch size and num_epochs:


In [ ]:
part1_batch_size = 64
part1_num_epochs = 30

Now you decide the rest.

Try changing the values for *num_layers* (number of transformer blocks), *d_model* (size of embedding and pooler layers), *d_kv* (size of query, key, and value vectors per attention head), *num_heads* (the number of attention heads), and *d_ff* (size of feed forward layers after each attention layer).

Find hyperparameters that finish training 30 epochs in 10-20 minutes on a free Colab T4 GPU, and that give you as low of a validation loss as you can, at least below 1.8. Also try to do this without overwhelming overfitting, i.e. try to keep training_loss / validation_loss > 0.6 after 30 epochs.

Then answer the questions below.

In [ ]:
"""
Define the values you want to use for d_model, d_kv, num_heads, and d_ff, for the T5Config below.
"""

### YOUR CODE HERE

# Define the values you want to use for d_model, d_kv, num_heads, and d_ff for the T5Config below.

### YOUR CODE HERE
embed_dim = 256
keyvalue_dim = 64
num_heads = 8
dense_dim = 1024

# Create and train the model
part1_model = create_from_scratch_model(num_layers, embed_dim, keyvalue_dim, dense_dim, num_heads)

# Fix decoder_start_token_id issue
part1_model.config.decoder_start_token_id = part1_model.config.pad_token_id

# Create training arguments
part1_training_args = create_seq2seq_training_args(part1_batch_size, part1_num_epochs)

# Create and train the model
part1_trainer = create_seq2seq_trainer(part1_model, part1_training_args, train_ds_part1, val_ds_part1)

# Train the model
part1_trainer.train()


### END YOUR CODE

Map:   0%|          | 0/16798 [00:00<?, ? examples/s]

Map:   0%|          | 0/1145 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,1.062100
1000,0.894700
1500,0.834800
2000,0.782200
2500,0.743800
3000,0.713000
3500,0.688000
4000,0.663600
4500,0.646300
5000,0.633200


TrainOutput(global_step=7890, training_loss=0.7070279197547825, metrics={'train_runtime': 3135.2169, 'train_samples_per_second': 160.735, 'train_steps_per_second': 2.517, 'total_flos': 4060440640880640.0, 'train_loss': 0.7070279197547825, 'epoch': 30.0})

In [ ]:
part1_model = create_from_scratch_model(num_layers, embed_dim, keyvalue_dim, dense_dim, num_heads)

part1_model.config.decoder_start_token_id = part1_model.config.pad_token_id

part1_training_args = create_seq2seq_training_args(part1_batch_size, part1_num_epochs)

part1_trainer = create_seq2seq_trainer(part1_model, part1_training_args, train_ds_part1, val_ds_part1)

part1_trainer.train()

Map:   0%|          | 0/16798 [00:00<?, ? examples/s]

Map:   0%|          | 0/1145 [00:00<?, ? examples/s]

Step,Training Loss
500,1.062100
1000,0.894700
1500,0.834800
2000,0.782200
2500,0.743800
3000,0.713000
3500,0.688000
4000,0.663600
4500,0.646300
5000,0.633200


TrainOutput(global_step=7890, training_loss=0.7070279197547825, metrics={'train_runtime': 3136.0704, 'train_samples_per_second': 160.692, 'train_steps_per_second': 2.516, 'total_flos': 4060440640880640.0, 'train_loss': 0.7070279197547825, 'epoch': 30.0})

**QUESTION:**

 1.a What is the final validation loss that you were able to achieve for the part1 model after training for 30 epochs? (Copy and paste the decimal value for the final validation loss, to 5 significant digits, e.g. a number like 0.56781 or 0.87632. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 1.b Which model config parameters (if any) did you increase, to achieve a lower validation loss, while staying within the training time and overfitting guidelines? (List the names of the parameters you increased, e.g. embed_dim, keyvalue_dim, num_heads, dense_dim, num_layers. Put this list in square brackets in the answers file.)

**QUESTION:**

 1.c Which model config parameters (if any) did you decrease, to achieve a lower validation loss, while staying within the training time and overfitting guidelines? (List the names of the parameters you decreased, e.g. embed_dim, keyvalue_dim, num_heads, dense_dim, num_layers. Put this list in square brackets in the answers file.)

In [ ]:
"""
Before moving on, save a checkpoint of the model you just trained in your Drive,
So that you can pick up where you left off later if needed
"""

# Modify this path to the location in your Drive where you want to save the part1 model
part1_model_checkpoint_filepath = 'drive/MyDrive/ISchool/MIDS/266/model_checkpoints/part1_model'

In [ ]:
# Run this line only after you've trained the part1 model
part1_model.save_pretrained(part1_model_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
part1_model = T5ForConditionalGeneration.from_pretrained(part1_model_checkpoint_filepath)

### 1.3: Text Generation Parameters

Cross-entropy loss is great for training, but it's not a very interpretable metric for manually reviewing how well the model is doing as we experiment with available options. Ultimately, we want to actually look at the translations the model outputs, compare them to human translations, and potentially judge other aspects of the actual output.

To do that, we need to actually generate some model output. Remember that the model itself predicts probabilities for each word in the vocabulary, based on what words have already been generated, at each decoder time-step. In order to select which actual words to output, there are multiple decoder strategies we can use that are build on top of the model's predicted probabilities. (E.g. beam search, top-k or top-p sampling, repeat ngram constraints, min/max length constraints, etc.)

Let's define a function below to generate translations for new inputs. Then we'll define another function to translate the validation set and calculate some standard evaluation metrics for translation, as well as print out some translations for manual inspection. We'll include some arguments that you'll experiment with next.

In [ ]:
def generate_output(model, tokenizer, input_sentences, batch_size, **kwargs):

    all_outputs = []

    for i in range(int(len(input_sentences) / batch_size) + 1):
        start_i, end_i = i * batch_size, (i + 1) * batch_size
        if start_i >= len(input_sentences):
            break

        inputs_encoded = tokenizer(input_sentences[start_i:end_i], padding=True, return_tensors='pt')
        output_ids = model.cuda().generate(inputs_encoded['input_ids'].cuda(), **kwargs)
        generated_sentences = tokenizer.batch_decode(output_ids,
                                                     skip_special_tokens=True,
                                                     clean_up_tokenization_spaces=False)
        all_outputs.extend(generated_sentences)

    return all_outputs

In [ ]:
# Load the BLEU metric and the trained BLEURT model for semantic similarity scoring

bleu = evaluate.load("bleu")

bleurt_tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
bleurt_model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def calculate_eval_metrics(text_pairs, model, tokenizer, batch_size, prefix="", **kwargs):
    original_texts = [prefix + pair[0] for pair in text_pairs]
    label_texts = [pair[1] for pair in text_pairs]

    # Translate original texts
    translations = generate_output(model, tokenizer, original_texts, batch_size, **kwargs)

    # Calculate BLEU scores
    bleu_results = bleu.compute(predictions=translations, references=label_texts)
    print('BLEU: ', bleu_results)

    # Calculate BLEURT scores
    bleurt_scores = []
    for i in range(int(len(translations) / batch_size) + 1):
        start_i, end_i = i * batch_size, (i + 1) * batch_size
        if start_i >= len(translations):
            break

        with torch.no_grad():
            scores = bleurt_model(**bleurt_tokenizer(label_texts[start_i:end_i],
                                                     translations[start_i:end_i],
                                                     truncation=True,
                                                     max_length=MAX_SEQUENCE_LENGTH,
                                                     padding='max_length',
                                                     return_tensors='pt'))[0].squeeze().numpy()
            if scores.shape:
                bleurt_scores.extend(scores)
            else:  # Happens when there was only one example in the last batch
                bleurt_scores.append(float(scores))

    print('BLEURT: ', np.mean(bleurt_scores))

    return translations

First, choose some keyword arguments to pass to the generate_output() function. These can be any parameters for the .generate() method (e.g. beam search or top-k or top-p sampling, no_repeat_ngram_size, etc). You will want to try the options listed in Question 1.e below, to be able to answer that question (but some of them can't be used at the same time). More info on each can be found in the [Huggingface documentation on text generation here](https://huggingface.co/docs/transformers/en/main_classes/text_generation).

Then run the function to translate the validation set and print out eval metrics. The function returns the translations, so we'll also print out a sample of those to manually inspect. Use what you see to iterate on the .generate() arguments, trying to find the most reasonable .generate() arguments that you can for the model you trained.

The output will not be great no matter what you do, but you should be able to make it a little more readable, with slightly better BLEU and BLEURT metrics, than the basic options specified in the Lesson 6 notebook.

Then answer the questions below.

In [ ]:
"""
Fill in the decoder .generate() arguments that you want to use, like num_beams or top_p, etc.
"""
# Ensure decoder_start_token_id is set
part1_model.config.decoder_start_token_id = part1_model.config.pad_token_id

part1_generate_kwargs = {
    "num_beams": 5,
    "top_p": 0.9,
    "min_length": 20,
    "max_length": 100,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "decoder_start_token_id": part1_tokenizer.pad_token_id
}

part1_val_translations = calculate_eval_metrics(
    val_pairs,
    part1_model,
    part1_tokenizer,
    part1_batch_size,
    **part1_generate_kwargs
)

BLEU:  {'bleu': 0.0, 'precisions': [0.23142414860681115, 0.021315192743764172, 0.0007237199203908088, 0.0], 'brevity_penalty': 0.4432610117077694, 'length_ratio': 0.5513905683192262, 'translation_length': 7752, 'reference_length': 14059}
BLEURT:  -1.1516432


In [ ]:
# Print out a sample of outputs to manually review
for i in range(10):
    sample_i = random.choice(range(len(part1_val_translations)))
    print('Original:    ', val_pairs[sample_i][0])
    print('Reference:   ', val_pairs[sample_i][1])
    print('Translation: ', part1_val_translations[sample_i])
    print()

Original:     Read on this book, That show of such an exercise may colour Your loneliness.
Reference:    Read this book, Showing that such an exercise may colours Your loneliness.
Translation:  Damn, Damn Damn

Original:     Say that she frown; I’ll say she looks as clear As morning roses newly washed with dew.
Reference:    If she glares, I’ll say her brow is as clear as roses newly washed with morning dew.
Translation:  right Ily Damn now Damn

Original:     Choose your revenge yourself.
Reference:    Choose your revenge.
Translation:  brook now

Original:     I shall return before your Lordship thence.
Reference:    I’ll be off before you are.
Translation:  May I right

Original:     No, sir, she shall not, though you take her part.
Reference:    No, sir, she shall not, although you take her side.
Translation:  now,, Damn,, Damn

Original:     give him a halter: I am famished in his service; you may tell every finger I have with my ribs.
Reference:    I am starving in his service; y

**QUESTION:**

 1.d What seems to be particularly bad about the part1 model's translations? (Choose one of the following options that you agree with most and put it in the answers file.)

 - A. The model keeps repeating the same common words or phrases over and over, which don't produce very meaningful statements.

 - B. The model is generating pretty good modern English, but it's quite offensive.

 - C. The model's output has mostly the same meaning as the input, but with minor grammatical mistakes.

 - D. The model is making up elaborate narrative details that don't appear in the original text.

**QUESTION:**

 1.e Which .generate() parameter seemed to help the most in addressing the main shortcoming(s) that you noticed in the part1 model's output? (Choose one of the following options and put it in the answers file.)

 - A. num_beams
 - B. do_sample
 - C. top_k
 - D. top_p
 - E. temperature
 - F. no_repeat_ngram_size

### 1.4 Test Set Evaluation Metrics

Once you've settled on training hyperparameters that produce good validation loss, and generation options that produce the best output you can so far, go ahead and calculate evaluation metrics on the test set, to warp up this from-scratch model.

Then answer the questions below.

In [ ]:
# Print out eval metrics for the part1_model on the test set

part1_test_translations = calculate_eval_metrics(
    test_pairs,
    part1_model,
    part1_tokenizer,
    part1_batch_size,
    **part1_generate_kwargs
)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


BLEU:  {'bleu': 0.0031810457757711803, 'precisions': [0.23375262054507337, 0.02091985848331026, 0.002402069475240207, 0.00022742779167614282], 'brevity_penalty': 0.4424700061334765, 'length_ratio': 0.5508480692890653, 'translation_length': 7632, 'reference_length': 13855}
BLEURT:  -1.1518452


**QUESTION:**

 1.f What is the overall BLEU score that you achieved on the test set for the part1 model? (Copy and paste the decimal value for the overall BLEU score, to 5 significant digits, e.g. a number like 0.03671 or 0.09763. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 1.g What is the mean BLEURT score that you achieved on the test set for the part1 model? (Copy and paste the decimal value for the mean BLEURT score, to 5 significant digits, e.g. a number like -1.12345 or -0.54321. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

## 2. Small Pre-Trained T5 Model

What if we use a model that has already been pre-trained to recognize English (at least modern English), even if it hasn't yet been trained for our particular translation task?

We'll use a T5 small model, which should be able to generate good modern English, but we'll need to train it to encode and translate Shakespearean text.

### 2.1 Pre-trained Model Setup and Tokenization

The next two cells load the pre-trained model, and preprocess the data with the pre-trained tokenizer. Fill in the necessary code for each of these cells.

For preprocessing, you'll need to map the `preprocess_translation_batch` function that we created earlier to the `train_dataset` and `val_dataset`. Use the code from part 1 as an example, but now pass in the pretrained T5 tokenizer as a function keyward argument (kwarg). Also pass in the given task_prefix as the "prefix" kwarg for the preprocessing function.

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the pre-trained model and tokenizer
t5_pretrained_checkpoint_name = 't5-small'  # Change from 'google/t5-small' to 't5-small'

### YOUR CODE HERE
part2_tokenizer = T5Tokenizer.from_pretrained(t5_pretrained_checkpoint_name)
part2_model = T5ForConditionalGeneration.from_pretrained(t5_pretrained_checkpoint_name)
### END YOUR CODE


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
"""
Preprocess the datasets using the pretrained tokenizer, and the given task_prefix.
Use the task_prefix as the "prefix" argument to the function preprocess_translation_batch().
"""

task_prefix = "Translate Shakespeare to Modern English: "

train_ds_part2 = train_dataset.map(
    preprocess_translation_batch,
    batched=True,
    fn_kwargs={
        "tokenizer": part2_tokenizer,
        "prefix": task_prefix
    }
)

val_ds_part2 = val_dataset.map(
    preprocess_translation_batch,
    batched=True,
    fn_kwargs={
        "tokenizer": part2_tokenizer,
        "prefix": task_prefix
    }
)

Map:   0%|          | 0/16798 [00:00<?, ? examples/s]

Map:   0%|          | 0/1145 [00:00<?, ? examples/s]

### 2.2 Fine-Tuning the Pre-Trained Model

Now create the training args and trainer to fine-tune this pre-trained model. We've given you part of the code: you'll use the same functions as above for `create_seq2seq_training_args` and `create_seq2seq_trainer`. Fill in the rest of the arguments that you need for this version of the model. Use the provided batch size and num_epochs.

In [ ]:
"""
Create the training args and trainer for the pre-trained model.
Use the batch size and num_epochs provided below for this model.
"""
from transformers import T5Tokenizer, Seq2SeqTrainer

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def create_seq2seq_trainer(model, training_args, train_ds, val_ds):

    def preprocess_translation_batch(batch):
        inputs = batch['shakespeare']
        targets = batch['modern']

        model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)
        labels = tokenizer(targets, padding="max_length", truncation=True, max_length=128)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Use train_ds and val_ds directly here as they are already preprocessed
    train_dataset = train_ds # No need to map again
    eval_dataset = val_ds # No need to map again

    # Create trainer
    # Pass in the newly created datasets: train_dataset and eval_dataset
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,  # Change from train_ds to train_dataset
        eval_dataset=eval_dataset   # Change from val_ds to eval_dataset
    )

    return trainer

Run the cell below to fine-tune the part2 model, then answer the following questions.

In [ ]:
part2_batch_size = 64
part2_num_epochs = 4

part2_training_args = create_seq2seq_training_args(part2_batch_size, part2_num_epochs)

part2_trainer = create_seq2seq_trainer(part2_model, part2_training_args, train_ds_part2, val_ds_part2)

**QUESTION:**

 2.a What is the final validation loss that you were able to achieve for the part2 model after training for 4 epochs? (Copy and paste the decimal value for the final validation loss, to 5 significant digits, e.g. a number like 0.56781 or 0.87632. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

In [ ]:
"""
Before moving on, save a checkpoint of the model you just trained in your Drive,
So that you can pick up where you left off later if needed
"""

# Modify this path to the location in your Drive where you want to save the part2 model
part2_model_checkpoint_filepath = 'drive/MyDrive/ISchool/MIDS/266/model_checkpoints/part2_model'

In [ ]:
# Run this line only after you've fine-tuned the part2_model
part2_model.save_pretrained(part2_model_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you fine-tuned earlier
part2_model = T5ForConditionalGeneration.from_pretrained(part2_model_checkpoint_filepath)

### 2.3 Fine-Tuned Model Evaluation

Now use the calculate_eval_metrics() function defined above to translate the test set and calculate evaluation metrics. Also print out a sample of the translated outputs. For now, use the same decoder .generate() kwargs that you chose for part1.

Then answer the questions below.

In [ ]:
# Print out eval metrics for the part2_model on the test set

part2_test_translations = calculate_eval_metrics(
    test_pairs,
    part2_model,
    part2_tokenizer,
    part2_batch_size,
    task_prefix,
    **part1_generate_kwargs
)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


BLEU:  {'bleu': 0.16929924458398787, 'precisions': [0.34086502314906414, 0.20040620077924232, 0.13276184674296157, 0.0905843561091775], 'brevity_penalty': 1.0, 'length_ratio': 1.823962468422952, 'translation_length': 25271, 'reference_length': 13855}
BLEURT:  -1.2141603


In [ ]:
# Print out a sample of the translated outputs to look at as well

for i in range(10):
    sample_i = random.choice(range(len(part2_test_translations)))
    print('Original:    ', test_pairs[sample_i][0])
    print('Reference:   ', test_pairs[sample_i][1])
    print('Translation: ', part2_test_translations[sample_i])
    print()

Original:     How now?
Reference:    What’s this?
Translation:  Translat Translat Translate Shakespeare Shakespeare to Modern English: How now? Translat Translater Shakespeare Shakespeare in Modern English.

Original:     The mighty gods defend thee!
Reference:    The mighty gods defend you!
Translation:  Translate Shakespeare Shakespeare to Modern English: The mighty gods defend thee! !

Original:     Then there were two cousins laid up, when the one should be lamed with reasons and the other mad without any.
Reference:    Then there would be two cousins lying sick in bed: one hurt by reasons and the other gone crazy without any.
Translation:  Translate Shakespeare Shakespeare to Modern English: Dann waren zwei cousins laid up, when the one should be lamed with reasons, and the other mad without any.

Original:     How say'st thou, that Macduff denies his person At our great bidding?
Reference:    What do you think about Macduff’s refusing to be At our great bidding?
Translation:  Tra

**QUESTION:**

 2.b What is the overall BLEU score that you achieved on the test set for the part2 model? (Copy and paste the decimal value for the overall BLEU score, to 5 significant digits, e.g. a number like 0.03671 or 0.09763. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 2.c What is the mean BLEURT score that you achieved on the test set for the part2 model? (Copy and paste the decimal value for the mean BLEURT score, to 5 significant digits, e.g. a number like -1.12345 or -0.54321. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 2.d What do you notice about the part2 model's output? It should be much better than the part1 model's output. But the translations still probably don't perfectly match the reference human translations. What does the part2 model seem to still be doing poorly? (Chose one of the following options that you agree with most, and put it in the answers file.)

 - A. The generated translations are gibberish.

 - B. The generated translations are written in a far more casual style than the reference human translations.

 - C. The generated translations mean something completely different from the input text and reference translations.

 - D. The generated translations are too similar to the input text, and haven't been rephrased as much as the reference human translations.

### 2.4 Style Classifier

Now that the model is able to output more coherent translations, we can start to get more picky about different aspects of the output. We should also make sure that our evaluation metrics are capturing everything we want to be able to assess and improve in the model's output.

One thing we're not capturing yet is if the output has the right **style**. This task is sort of a translation task, but since it's between two different forms of English, we can also think of it as a style transfer task.

BLEU might help a little with that, but when the model chooses different words from the human reference, it could do so in ways that are still good modern English or that are still too much like Shakespeare. BLEURT won't tell us anything about the style, as long as the meaning is still similar to the reference.

How can we tell whether the output has the right style? We could train a separate classification model to predict whether text is Shakespearean or modern English. We have the data to do it! We just need to repurpose our data for a classification problem.

Use the code below to train a BERT classifier to predict whether a sentence is Shakespearean or modern English. We're providing this code for you, because it's not the main task and not based on a similar example from class. We want you to use it as one of your evaluation metrics, to help you iterate on your models for the main task.

In [ ]:
def make_style_classifier_data(text_pairs):
    style_texts = [pair[0] for pair in train_pairs] + [pair[1] for pair in train_pairs]
    style_labels = [0 for pair in train_pairs] + [1 for pair in train_pairs]

    style_dataset = Dataset.from_dict({"text": style_texts, "label": style_labels})

    return style_dataset.shuffle()

style_train_ds = make_style_classifier_data(train_pairs)
style_valid_ds = make_style_classifier_data(val_pairs)

In [ ]:
bert_checkpoint_name = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_checkpoint_name)
bert_style_classifier_model = BertForSequenceClassification.from_pretrained(bert_checkpoint_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_style_text(data):
    return bert_tokenizer.batch_encode_plus(
            data['text'],
            max_length=MAX_SEQUENCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )

style_train_ds_preprocessed = style_train_ds.map(preprocess_style_text, batched=True)
style_valid_ds_preprocessed = style_valid_ds.map(preprocess_style_text, batched=True)

Map:   0%|          | 0/33596 [00:00<?, ? examples/s]

Map:   0%|          | 0/33596 [00:00<?, ? examples/s]

In [ ]:
style_classifier_batch_size = 32
style_classifier_num_epochs = 2

style_training_args = TrainingArguments(
    output_dir="bert_shakespeare_style_classifier",
    per_device_train_batch_size=style_classifier_batch_size,
    per_device_eval_batch_size=style_classifier_batch_size,
    num_train_epochs=style_classifier_num_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to='none'
)

In [ ]:
metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
style_trainer = Trainer(
    model=bert_style_classifier_model,
    args=style_training_args,
    train_dataset=style_train_ds_preprocessed,
    eval_dataset=style_valid_ds_preprocessed,
    compute_metrics=compute_metrics
)

In [ ]:
style_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.409100,0.283811,0.863347
2,0.294700,0.218208,0.892963


TrainOutput(global_step=2100, training_loss=0.36151477177937824, metrics={'train_runtime': 634.9336, 'train_samples_per_second': 105.825, 'train_steps_per_second': 3.307, 'total_flos': 1381168596230400.0, 'train_loss': 0.36151477177937824, 'epoch': 2.0})

In [ ]:
"""
Before moving on, save a checkpoint of the model you just trained in your Drive,
So that you can pick up where you left off later if needed
"""

# Modify this path to the location in your Drive where you want to save the style classifier
style_classifier_checkpoint_filepath = 'drive/MyDrive/ISchool/MIDS/266/model_checkpoints/style_classifier'

In [ ]:
# Run this line only after you've trained the style classifier model
bert_style_classifier_model.save_pretrained(style_classifier_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the style classifier you trained earlier
bert_style_classifier_model = BertForSequenceClassification.from_pretrained(style_classifier_checkpoint_filepath)

Now let's use the style classifier to classify the output from the Shakespeare translation model, using the test set from our main task. The function reports the average predicted probability of the positive class, which is the modern English style (and which is our goal for our main task model).

We should also classify the original Shakespearean text and the human translations from the test set, to compare the scores as references.

Run the next two cells of code, then answer the following questions.

In [ ]:
def get_modern_style_score(text):
  text_inputs = bert_tokenizer.batch_encode_plus(
            text,
            max_length=MAX_SEQUENCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )

  with torch.no_grad():
      logits = bert_style_classifier_model.cuda()(text_inputs['input_ids'].cuda(),
                                                  attention_mask=text_inputs['attention_mask'].cuda()).logits

  probs = softmax(logits.cpu().numpy(), axis=1)
  return np.mean(probs[:, 1])

In [ ]:
test_original_texts = [task_prefix + pair[0] for pair in test_pairs]
test_label_texts = [pair[1] for pair in test_pairs]

translations_score = get_modern_style_score(part2_test_translations)
reference_score = get_modern_style_score(test_label_texts)
shakespeare_score = get_modern_style_score(test_original_texts)

print("Modern style score for generated translations:  ", translations_score)
print("Modern style score for reference translations:  ", reference_score)
print("Modern style score for original Shakespeare:    ", shakespeare_score)

Modern style score for generated translations:   0.20933953
Modern style score for reference translations:   0.81587267
Modern style score for original Shakespeare:     0.28676507


**QUESTION:**

 2.e What is the modern style classifier score that you got for the part2 model's generated translations? (Copy and paste the decimal value from the get_modern_style_score function above, to 5 significant digits, e.g. a number like 0.36712 or 0.97632. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 2.f What is the modern style classifier score that you got for the human reference translations? (Copy and paste the decimal value from the get_modern_style_score function above, to 5 significant digits, e.g. a number like 0.36712 or 0.97632. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 2.g What is the modern style classifier score that you got for the original Shakespeare text? (Copy and paste the decimal value from the get_modern_style_score function above, to 5 significant digits, e.g. a number like 0.36712 or 0.97632. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 2.h What do you notice about differences between these scores, and what does that tell you about what the part2 model is doing? (Chose one of the following options that you agree with most, and put it in the answers file.)

 - A. The part2 model is generating output that is way more modern, casual, and younger generation speak than the human translations.

 - B. The part2 model is generating output that looks about as modern as the human translations, even if it doesn't always mean the same thing.

 - C. The part2 model is generating output that is partly modernized, more modern than the original Shakespeare, but still not as modern as the human references.

 - D. The part2 model is generating output that is still pretty much the same style as the original Shakespeare text.

### 2.5 Revisit Decoder .Generate() Options

Now that we have one more evaluation metrics, let's go back to the decoder .generate() arguments we used before. Are there any arguments you want to change, to try to do better on this latest evaluation metric?

Try different options for the part2_generate_kwargs below, and run the two cells afterward with each set of choices to see how the evaluation metrics change.

Then answer the questions below.

In [ ]:
"""
Fill in the decoder .generate() arguments that you want to use for the part2 model, like num_beams or top_p, etc.
"""

part2_generate_kwargs = {

    ### YOUR CODE HERE
    "num_beams": 5,
    "top_p": 0.9,
    "min_length": 20,
    "max_length": 100,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
}


    ### END YOUR CODE

In [ ]:
# Print out eval metrics for the part2_model on the test set, with the new kwargs

part2_test_translations = calculate_eval_metrics(
    test_pairs,
    part2_model,
    part2_tokenizer,
    part2_batch_size,
    task_prefix,
    **part2_generate_kwargs
)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


BLEU:  {'bleu': 0.16929924458398787, 'precisions': [0.34086502314906414, 0.20040620077924232, 0.13276184674296157, 0.0905843561091775], 'brevity_penalty': 1.0, 'length_ratio': 1.823962468422952, 'translation_length': 25271, 'reference_length': 13855}
BLEURT:  -1.2141603


In [ ]:
# Calculate modern style scores for the part2 translations after using the new kwargs

translations_score = get_modern_style_score(part2_test_translations)

print("Modern style score for generated translations:  ", translations_score)

Modern style score for generated translations:   0.20933953


In [ ]:
# Print out a sample of the translated outputs with the revised .generate() parameters

for i in range(10):
    sample_i = random.choice(range(len(part2_test_translations)))
    print('Original:    ', test_pairs[sample_i][0])
    print('Reference:   ', test_pairs[sample_i][1])
    print('Translation: ', part2_test_translations[sample_i])
    print()

Original:     How now?
Reference:    What’s this?
Translation:  Translat Translat Translate Shakespeare Shakespeare to Modern English: How now? Translat Translater Shakespeare Shakespeare in Modern English.

Original:     The mighty gods defend thee!
Reference:    The mighty gods defend you!
Translation:  Translate Shakespeare Shakespeare to Modern English: The mighty gods defend thee! !

Original:     Then there were two cousins laid up, when the one should be lamed with reasons and the other mad without any.
Reference:    Then there would be two cousins lying sick in bed: one hurt by reasons and the other gone crazy without any.
Translation:  Translate Shakespeare Shakespeare to Modern English: Dann waren zwei cousins laid up, when the one should be lamed with reasons, and the other mad without any.

Original:     How say'st thou, that Macduff denies his person At our great bidding?
Reference:    What do you think about Macduff’s refusing to be At our great bidding?
Translation:  Tra

**QUESTION:**

 2.i Which decoder strategy seemed to increase the modern style score the most? (Choose one of the following options and put it in the answers file.)

 - A. Using a stricter option to always choose the highest predicted possibility output (e.g. beam search, or small k or p when using sampling).

 - B. Using a looser sampling method to allow the model to choose more varied output (e.g. top-k or top-p rather than beam search, especially with higher k or p and/or higher temperature).

**QUESTION:**

 2.j What happens to the other evaluation metrics when you try to increase the modern style score by varying the decoder strategy discussed in 2.i? (Choose one of the following options and put it in the answers file.)

 - A. BLEU and BLEURT both seem to be positively correlated with the modern style score, when changing the decoder strategy.

 - B. BLEU and BLEURT both seem to be negatively correlated with the modern style score, when changing the decoder strategy.

 - C. BLEU seems to move with the modern style score, but BLEURT seems to go the other direction.

 - D. BLEURT seems to move with the modern style score, but BLEU seems to go the other direction.

**QUESTION:**

 2.k Why do you think the relationship in question 2.j is happening? (Choose one of the following options and put it in the answers file.)

 - A. A stricter decoder strategy makes the model more likely to output the best translation, which is good for BLEU, BLEURT, and modern style objectives.

 - B. A looser decoder strategy gives the model more freedom to find a good modern style translation, which should also end up saying more of the same things in the same way as the human translation.

 - C. A stricter decoder strategy makes the model more likely to output a translation that has correct exact words and style, increasing BLEU and modern style scores, but might not mean the same thing as the human translation.

 - D. A looser decoder strategy gives the model more freedom to choose more modern style words, which the pre-trained model is more familiar with, but that freedom can make the model less likely to end up with the exact same words or meaning as the human translation.

 - E. A stricter decoder strategy makes the model more likely to choose more of the exact same words as used in the dataset, but not necessarily in the same order, so the meaning and style don't end up being as close to the human translation.

## 3. Adding Supplementary Paraphrase Dataset

Can we do anything else to make the model capable of rephrasing the input text more into a different style (i.e. modernizing it more fully away from the Shakespeare), but still keep the same meaning?

One related task that could help is a paraphrasing task. The [GLUE Microsoft Research Paraphrase Corpus (MRPC)](https://huggingface.co/datasets/nyu-mll/glue) dataset has pairs of sentences with labels indicating whether the two sentences are equivalent (i.e. they mean the same thing) or not.

We could use that data as an additional supporting task for our T5 model, to see if it helps our model get better at accurately rephrasing the input text into a differently worded output.

### 3.1 Load and preprocess the supplemental dataset

Load the dataset from Huggingface and look at the contents

In [ ]:
mrpc_data = load_dataset('nyu-mll/glue', 'mrpc')

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
mrpc_data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
mrpc_data['train'].features['label'].names

['not_equivalent', 'equivalent']

Let's use just the pairs that are labeled as equivalent (correct paraphrases), and split the sentences to use the first as the model's input and second as the model's output. Then we can use that to train our T5 model to better generate rephrased statements in modern English with the same meaning as the input but in different words.

In [ ]:
mrpc_equiv_train = mrpc_data['train'].filter(lambda example: example['label'] == 1)
mrpc_equiv_valid = mrpc_data['validation'].filter(lambda example: example['label'] == 1)

Filter:   0%|          | 0/3668 [00:00<?, ? examples/s]

Filter:   0%|          | 0/408 [00:00<?, ? examples/s]

Fill in the code below to encode "sentence1" as the model's input and "sentence2" as the model's output.

We will also add a different prefix for this supporting task, so make sure to add the prefix to the inputs in the function below. (You can use the preprocess_translation_batch function above as an example.)

In [ ]:
def preprocess_mrpc_for_paraphrase_generation(mrpc_ds, tokenizer, prefix):
    ### YOUR CODE HERE ###

    # Directly access the columns instead of iterating through examples
    inputs = [prefix + sentence for sentence in mrpc_ds["sentence1"]]
    outputs = [sentence for sentence in mrpc_ds["sentence2"]]

    input_encoded = tokenizer(inputs, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    output_encoded = tokenizer(outputs, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    ### END YOUR CODE ###

    return {'input_ids': input_encoded['input_ids'],
            'labels': output_encoded['input_ids']}

Now map the preprocessing function to the MRPC train and validation datasets. Use the part2_tokenizer to preprocess the data, since we're using the same T5 pre-trained model checkpoint as in part 2. For the preprocessing function's "prefix" argument, use the paraphrase_prefix provided below.

In [ ]:
paraphrase_prefix = 'Paraphrase in modern English: '

### YOUR CODE HERE ###

mrpc_paraphrase_train = mrpc_equiv_train.map(
    lambda batch: preprocess_mrpc_for_paraphrase_generation(batch, part2_tokenizer, paraphrase_prefix),
    batched=True
)

mrpc_paraphrase_valid = mrpc_equiv_valid.map(
    lambda batch: preprocess_mrpc_for_paraphrase_generation(batch, part2_tokenizer, paraphrase_prefix),
    batched=True
)

### END YOUR CODE ###


Map:   0%|          | 0/2474 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

### 3.2 Train T5 on Paraphrasing Task

Load a fresh copy of the pre-trained T5 model (using the same pre-trained model checkpoint as part2), so that we can train it first on the paraphrase task, and last on the main task data.

In [ ]:
from transformers import T5ForConditionalGeneration

# Load a new copy of the same pre-trained model (we'll use the same tokenizer as part2)
t5_pretrained_checkpoint_name = 'google-t5/t5-small'

### YOUR CODE HERE ###

part3_model = T5ForConditionalGeneration.from_pretrained(t5_pretrained_checkpoint_name)

### END YOUR CODE ###


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Now create the training args and trainer for the paraphrase task, and train the model. Use the `create_seq2seq_training_args` and `create_seq2seq_trainer` functions like before.

You'll be using the part3_model you just loaded, and the MRPC data you preprocessed. Use the batch_size and num_epochs provided for the paraphrase task below.

In [ ]:
# Remove 'idx' column from dataset before training
mrpc_paraphrase_train = mrpc_paraphrase_train.remove_columns(["idx"])
mrpc_paraphrase_valid = mrpc_paraphrase_valid.remove_columns(["idx"])

paraphrase_training_args = create_seq2seq_training_args(
    batch_size=paraphrase_batch_size,
    num_epochs=paraphrase_num_epochs
)

paraphrase_trainer = create_seq2seq_trainer(
    model=part3_model,
    training_args=paraphrase_training_args,
    train_ds=mrpc_paraphrase_train,
    val_ds=mrpc_paraphrase_valid
)

In [ ]:
paraphrase_trainer.train()

Step,Training Loss


TrainOutput(global_step=312, training_loss=0.5838145720653045, metrics={'train_runtime': 176.0324, 'train_samples_per_second': 56.217, 'train_steps_per_second': 1.772, 'total_flos': 334835616841728.0, 'train_loss': 0.5838145720653045, 'epoch': 4.0})

### 3.3 Fine-Tune Paraphrase-Trained Model on Main Task

Now create the training args and trainer for the main task. Use the `create_seq2seq_training_args` and `create_seq2seq_trainer` functions one more time.

You'll be using the same model that you just trained on the paraphrase task (part3_model). Use the batch size and num epochs provided below.

For training data, use the same data as part2: `train_ds_part2` and `val_ds_part2`. We're using the same pre-trained model checkpoint, i.e. the same tokenizer, and the same task prefix, so the data has already been preprocessed correctly.

In [ ]:
"""
Create the training args and trainer for the main task using the part3_model.
"""

part3_batch_size = 32
part3_num_epochs = 4

### YOUR CODE HERE ###

# Define training arguments
part3_training_args = create_seq2seq_training_args(
    batch_size=part3_batch_size,
    num_epochs=part3_num_epochs
)

# Create the trainer
part3_trainer = create_seq2seq_trainer(
    model=part3_model,
    training_args=part3_training_args,
    train_ds=train_ds_part2,
    val_ds=val_ds_part2
)

### END YOUR CODE ###

In [ ]:
part3_trainer.train()

Step,Training Loss
500,0.826200
1000,0.736200
1500,0.732900
2000,0.712600


TrainOutput(global_step=2100, training_loss=0.7504303232828776, metrics={'train_runtime': 386.0025, 'train_samples_per_second': 174.071, 'train_steps_per_second': 5.44, 'total_flos': 710459869102080.0, 'train_loss': 0.7504303232828776, 'epoch': 4.0})

In [ ]:
"""
Before moving on, save a checkpoint of the model you just trained in your Drive,
So that you can pick up where you left off later if needed
"""

# Modify this path to the location in your Drive where you want to save the part3 model
part3_model_checkpoint_filepath = 'drive/MyDrive/ISchool/MIDS/266/model_checkpoints/part3_model'

In [ ]:
# Run this line only after you've trained the part3 model on both tasks
part3_model.save_pretrained(part3_model_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the part3 model you trained earlier
part3_model = T5ForConditionalGeneration.from_pretrained(part3_model_checkpoint_filepath)

### 3.4 Paraphrase-Trained Model Evaluation

Use the functions defined above to translate the test set and calculate the same set of evaluation metrics as used on the part2 model.

Use the same decoder .generate() arguments as part2 (`part2_generate_kwargs`), so that we can compare the part2 and part3 models as closely as possible.

Run the next three cells, then answer the questions below.

In [ ]:
# Print out eval metrics for the part3_model on the test set, with the new kwargs

part3_test_translations = calculate_eval_metrics(
    test_pairs,
    part3_model,
    part2_tokenizer,
    part3_batch_size,
    task_prefix,
    **part2_generate_kwargs
)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


BLEU:  {'bleu': 0.218508781585357, 'precisions': [0.43290503333011304, 0.25320856982154727, 0.17124701281772756, 0.12144553193953785], 'brevity_penalty': 1.0, 'length_ratio': 1.494189823168531, 'translation_length': 20702, 'reference_length': 13855}
BLEURT:  -0.5363374


In [ ]:
# Calculate modern style scores for the part3 translations after using the new kwargs

translations_score = get_modern_style_score(part3_test_translations)

print("Modern style score for generated translations:  ", translations_score)

Modern style score for generated translations:   0.7410207


In [ ]:
# Print out a sample of the translated outputs to look at as well

for i in range(10):
    sample_i = random.choice(range(len(part3_test_translations)))
    print('Original:    ', test_pairs[sample_i][0])
    print('Reference:   ', test_pairs[sample_i][1])
    print('Translation: ', part3_test_translations[sample_i])
    print()

Original:     How now?
Reference:    What’s this?
Translation:  How do I get to this point of the point of view of the world? – What’s the point?

Original:     The mighty gods defend thee!
Reference:    The mighty gods defend you!
Translation:  The mighty gods defend you! I’m sure you’re going to be a god!

Original:     Then there were two cousins laid up, when the one should be lamed with reasons and the other mad without any.
Reference:    Then there would be two cousins lying sick in bed: one hurt by reasons and the other gone crazy without any.
Translation:  Then there were two cousins laid up, when the one should be lamed with reasons and the other mad without any.

Original:     How say'st thou, that Macduff denies his person At our great bidding?
Reference:    What do you think about Macduff’s refusing to be At our great bidding?
Translation:  How do you say that Macduff denies his person At our great bidders?

Original:     This is a scurvy tune too: but here's my comfort.
Re

**QUESTION:**

 3.a What is the overall BLEU score that you achieved on the test set for the part3 model? (Copy and paste the decimal value for the overall BLEU score, to 5 significant digits, e.g. a number like 0.03671 or 0.09763. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 3.b What is the mean BLEURT score that you achieved on the test set for the part3 model? (Copy and paste the decimal value for the mean BLEURT score, to 5 significant digits, e.g. a number like -1.12345 or -0.54321. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 3.c What is the modern style classifier score that you got for the part3 model's generated translations? (Copy and paste the decimal value from the get_modern_style_score function above, to 5 significant digits, e.g. a number like 0.36712 or 0.97632. Put the answer in the answers file; it should match the value shown in your output in this notebook.)

**QUESTION:**

 3.d How do the part3 model's evaluation scores and output compare to the part2 model? Write a short answer about what you observe in the markdown cell below.

*** YOUR ANSWER TO QUESTION 3.d HERE *** The part3 model shows improvements in modern style score (0.74102 compared to part2).
However, the BLEU score (0.21859) and BLEURT score (-0.53633) suggest that the model may not be perfectly aligned with the reference translations.

*** END YOUR ANSWER ***